In [2]:
import serial
import time
import threading
from datetime import datetime

# Configuración del puerto serial
arduino_port = 'COM5'  # Cambia esto si usas otro puerto
baud_rate = 9600
ser = serial.Serial(arduino_port, baud_rate, timeout=1)
time.sleep(2)  # Esperar que el Arduino se reinicie

# Variable de control
running = True

# Función para enviar comandos al Arduino
def send_command(command):
    ser.write(f"{command}\n".encode())
    time.sleep(0.5)
    response = ser.read(ser.in_waiting).decode('utf-8')
    return response

# Hilo para lectura de entrada del usuario
def user_input_thread():
    global running
    while running:
        user_input = input("Ingresa un comando ('SET_STEPS 35-10,50-20,60-30' o 'stop'): ").strip()
        if user_input.lower() == "stop":
            running = False
        elif user_input.startswith("SET_STEPS"):
            # Validación básica del formato
            if "-" in user_input and "," in user_input:
                response = send_command(user_input)
                print(f"Respuesta de Arduino: {response}")
            else:
                print("Formato incorrecto. Usa: SET_STEPS 60-10,64-15,68-20")
        elif user_input == "SEND_DATA":
            response = send_command("SEND_DATA")
            print("Iniciando transmisión de datos...")
        else:
            print("Comando no reconocido.")

# Iniciar hilo de entrada del usuario
thread = threading.Thread(target=user_input_thread)
thread.start()

# Generar nombre de archivo con fecha y hora
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f"maceracion_datos_{timestamp}.csv"


# Crear archivo CSV con encabezado
with open(filename, "w") as file:
    file.write("Tiempo(s),Temperatura(°C),Setpoint(°C),Output(°C),Potencia(W),pH\n")

    # Leer datos del Arduino en loop
    while running:
        try:
            line = ser.readline().decode('utf-8').strip()
            if line:
                if line == "%%END%%":
                    print("\n✅ Transmisión finalizada por Arduino.")
                    print(f"📁 Archivo '{filename}' generado correctamente.")
                    print("⏱️ Todos los intervalos definidos se han completado.\n")
                    running = False  # Detener la lectura
                    break  # Salir del loop inmediatamente

                print(">>", line)
                file.write(line + "\n")
        except Exception as e:
            print(f"Error al leer datos: {e}")

ser.close()


Ingresa un comando ('SET_STEPS 35-10,50-20,60-30' o 'stop'):  SET_STEPS 35-10,50-20,60-10


Respuesta de Arduino: 


Ingresa un comando ('SET_STEPS 35-10,50-20,60-30' o 'stop'):  SEND_DATA


Iniciando transmisión de datos...
>> 27,26.07,0.00,0.00,0.00,16.77
>> 28,26.07,0.00,0.00,0.00,16.90
>> 29,26.07,0.00,0.00,0.00,17.04
>> 30,26.11,0.00,0.00,0.00,16.68
>> 32,26.07,0.00,0.00,0.00,17.33


Ingresa un comando ('SET_STEPS 35-10,50-20,60-30' o 'stop'):  SET_STEPS 35-10,50-20,60-30


Respuesta de Arduino: 
>> 33,26.07,0.00,0.00,0.00,17.34
>> 34,26.07,0.00,0.00,0.00,16.95
>> 35,26.11,0.00,0.00,0.00,16.84
>> 36,26.04,0.00,0.00,0.00,16.74
>> 37,26.11,0.00,0.00,0.00,16.74
>> 39,26.07,0.00,0.00,0.00,16.82
>> 40,26.11,0.00,0.00,0.00,16.75
>> 41,26.04,0.00,0.00,0.00,16.84
>> 42,26.07,0.00,0.00,0.00,16.78
>> 43,26.04,0.00,0.00,0.00,17.15
>> 44,26.11,0.00,0.00,0.00,16.93
>> 46,26.07,0.00,0.00,0.00,16.98
>> 47,26.11,0.00,0.00,0.00,17.06
>> 48,26.04,0.00,0.00,0.00,17.45
>> 49,26.11,0.00,0.00,0.00,17.15
>> 50,26.07,0.00,0.00,0.00,16.75
>> 52,26.07,0.00,0.00,0.00,16.77
>> 53,26.11,0.00,0.00,0.00,16.83
>> 54,26.07,0.00,0.00,0.00,16.77
>> 55,26.07,0.00,0.00,0.00,16.84
>> 56,26.11,0.00,0.00,0.00,17.60
>> 57,26.11,0.00,0.00,0.00,16.74
>> 59,26.11,0.00,0.00,0.00,16.75
>> 60,26.11,0.00,0.00,0.00,16.88
>> 61,26.11,0.00,0.00,0.00,16.98
>> 62,26.11,0.00,0.00,0.00,16.86
>> 63,26.07,0.00,0.00,0.00,17.18
>> 65,26.11,0.00,0.00,0.00,17.42
>> 66,26.11,0.00,0.00,0.00,16.98
>> 67,26.07,0.00,0.0

Ingresa un comando ('SET_STEPS 35-10,50-20,60-30' o 'stop'):  stop


>> 87,26.14,0.00,0.00,0.00,17.02
